In [1]:
from Bio import AlignIO
import os

In [2]:
cadherin_msa_path = os.path.join('..', 'data', 'cadherin', 'PF00028.alignment.seed')
alignment = AlignIO.read(cadherin_msa_path, 'stockholm')

In [3]:
seqs = []

for record in alignment:
    seqs.append(str(record.seq).replace('-', ''))
    # seqs.append(record.seq)


In [4]:
seqs[0]

'QTVRIKENVPVGTKTIGYKAYDPETGSSSGIRYKKSSDPEGWVDVDKNSGVITILKRLDREARSGVYNISIIASDKDGRTCNGVLGIVLE'

In [5]:
seqs

['QTVRIKENVPVGTKTIGYKAYDPETGSSSGIRYKKSSDPEGWVDVDKNSGVITILKRLDREARSGVYNISIIASDKDGRTCNGVLGIVLE',
 'KVIQSEDCLPTGTELLGYKAVDPERGTGEGLRYKKIQDEDNWFEINEYTGDLKTVKVLDRESTFVKNNQYNVSVIAFDADGRSCTGTLVVFLE',
 'KIIRQEEGLHAGTVLTTFTAQDPDRYMQQNIRYTKLSDPANWLKIDSVNGQITTIAVLDRESPNVKANIYNATFLASDNGIPPMSGTGTLQIYLL',
 'LRTSIAEGAPPGTSVATFSARDPDTEQLQRISYSKDYDPEDWLQVDGATGRIQTQRVLSPASPFLKDGWYRAIILALDNAIPPSTATGTLSIEIL',
 'RLAQVPEDVPLGQPLASYTAQDPDRAQQQRIKYVMGSDPAGWLAVHPENGLITAREQLDRESPFTKNSTYMAVLLAVDDGLPPATGTGTLLLTLL',
 'IVRQIKEGIPLKTPIVTVTADDPDSGLNGKVSYAISKQEPQLPQGRHFGINTETGVIHTLREIDRESIDTFRLTVVATDRAQPSERQLSTEKLVTVIVE',
 'YLARVMENTVPPNGGYVLTVNAYDADTPPLNSQVRYFLKEGDSDLFRINASSGDIALLKPLDREQQSEYTLTLVAMDTGSPPLTGTGIVRVEVQ',
 'YNVSLNETALPGTPVVTVMASDNDLGDNSKITYYLAETEHQFTVNPETGVISTTERVNCPQQTNVKSSASQKSCVFTVFARDHGSPRQDGRTYVTVNLL',
 'YSAVLSELAPTGSFVASITATDEDTGVNAQVHYDILSGNELKWFSMDPLTGLIVTTGPLDREIRDTVELSISARDGGPNPKFAYTQLKVIIL',
 'IAISFSESATSGTRLLLDAATDADVGENGVTDQYEIVAGNVDNKFRLVTTANPSGDTSYLHLETTGNLDRESRGSYQLNISARDGGSPPRFGYLQVNVTIL

In [6]:
from transformers import AutoTokenizer, EsmModel

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t36_3B_UR50D")
model = EsmModel.from_pretrained("facebook/esm2_t36_3B_UR50D")
model = model.to('cuda')

/nfshomes/vla/scratch/micromamba/envs/protein-lm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  6.53it/s]
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t36_3B_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
inputs = tokenizer([seqs[0], seqs[1]], padding=True, add_special_tokens=True,return_tensors = 'pt').to('cuda')

In [10]:
inputs['input_ids'].shape

torch.Size([2, 95])

In [30]:
import torch
with torch.no_grad():
    predicted_contacts = model.predict_contacts(inputs['input_ids'], inputs['attention_mask']).cpu().numpy()

In [31]:
predicted_contacts.shape

(2, 93, 93)

In [32]:
predicted_contacts[0, :len(seqs[0]), :len(seqs[0])].shape

(90, 90)

In [33]:
predicted_contacts[1, :len(seqs[1]), :len(seqs[1])].shape

(93, 93)

In [25]:
predicted_contacts.max()

tensor(1.0000, device='cuda:0', grad_fn=<MaxBackward1>)

In [26]:
predicted_contacts.min()

tensor(1.3161e-21, device='cuda:0', grad_fn=<MinBackward1>)

In [19]:
import sys
sys.path.append('..')

from scripts.esm_inference import SequenceDataset

In [21]:
dataset = SequenceDataset(cadherin_msa_path)

In [22]:
from torch.utils.data import DataLoader
eval_dataloader = DataLoader(dataset, batch_size =16)

In [24]:
for batch in eval_dataloader:
    b = batch
    break

In [27]:
for i,seq in enumerate(b):
    print(len(seq))

90
93
95
95
95
99
94
99
92
101
96
94
94
91
94
95
